# Minikube

> `minikube` is a minimal release of Kubernetes which allows us to create mini-cluster on `localhost`

This version is useful to:
- learn and experiment with Kubernetes
- develop applications locally (`dev` centered approach)

Minimal requirements:
- 2 CPUs
- 2 GB free RAM memory
- 20 GB free disk space
- Container or VM (previously setup `docker` is fine)

> `minikube` will use docker container (or VM) to imitate single working node on your `localhost`


## Installation

Please refer to [installation](https://minikube.sigs.k8s.io/docs/start/) in the documentation.

For Linux:
- Use package manager of your distribution
- Or run the following shell script:

In [ ]:
curl -LO https://storage.googleapis.com/minikube/releases/latest/minikube-linux-amd64
sudo install minikube-linux-amd64 /usr/local/bin/minikube

As Kubernetes (and `minikube` also) is written in `golang` it has the following features:
- Compiled binary: __download appropriate version and it is ready to run__
- Static linking: no folders, structures or dependencies, __everything is contained within the binary__

In [ ]:
minikube --help

In order to start `kubernetes` server a simple command can be used:

In [ ]:
minikube start

We can also start `kubernetes` dashboard which allows us to:
- monitor our deployment
- monitor status of the cluster
- monitor storage and persistent volumes
- check secrets provided 

After selecting `all namespaces` at the top bar we should see something along these lines:

![](./images/kubernetes-dashboard.png)

# kubectl

> __`kubectl` allows us to interact with `cluster` (e.g. deploy our configs, monitor PODs etc.)__

While `minikube` offers a way to use `kubectl` (and download appropriate version) we will install it separately:

### Installation

Check out the following [link](https://kubernetes.io/docs/tasks/tools/#kubectl) and choose appropriate OS.

For `Linux`, one can also download it via `curl` (still `golang` distributed binary) and install with `root` permissions using two commands below:

In [ ]:
# Download
curl -LO "https://dl.k8s.io/release/$(curl -L -s https://dl.k8s.io/release/stable.txt)/bin/linux/amd64/kubectl"

In [ ]:
# Installation with `sudo`
sudo install -o root -g root -m 0755 kubectl /usr/local/bin/kubectl

> __`kubectl` should also be available via package managers on most major OSes and Linux distros__

Verify `kubectl` is available by running the cell below (use `!` if you don't have `bash` kernel for `jupyter` notebook):

In [1]:
kubectl --help

kubectl controls the Kubernetes cluster manager.

 Find more information at:
https://kubernetes.io/docs/reference/kubectl/overview/

Basic Commands (Beginner):
  create        Create a resource from a file or from stdin.
  expose        Take a replication controller, service, deployment or pod and
expose it as a new Kubernetes Service
  run           Run a particular image on the cluster
  set           Set specific features on objects

Basic Commands (Intermediate):
  explain       Documentation of resources
  get           Display one or many resources
  edit          Edit a resource on the server
  delete        Delete resources by filenames, stdin, resources and names, or by
resources and label selector

Deploy Commands:
  rollout       Manage the rollout of a resource
  scale         Set a new size for a Deployment, ReplicaSet or Replication
Controller
  autoscale     Auto-scale a Deployment, ReplicaSet, StatefulSet, or
ReplicationController

Cluster Management Commands:
  certifi

: 1

## `kubectl` syntax

In general, when using `kubectl` the following could be used:

In [ ]:
kubectl [command] [TYPE] [NAME] [flags]

where:

- `command` - specifies operation one wants to perform (e.g. `get`, `describe`)
- `TYPE` - specifies resource type; __case in-sensitive, works with abbreviations and plurals__

For example, these commands do the same thing:

In [2]:
kubectl get pod
kubectl get pods
kubectl get po

NAME                                READY   STATUS    RESTARTS   AGE
nginx-deployment-66b6c48dd5-8z2j9   1/1     Running   1          5d
nginx-deployment-66b6c48dd5-bd2lq   1/1     Running   1          5d
nginx-deployment-66b6c48dd5-jv8zh   1/1     Running   1          5d
NAME                                READY   STATUS    RESTARTS   AGE
nginx-deployment-66b6c48dd5-8z2j9   1/1     Running   1          5d
nginx-deployment-66b6c48dd5-bd2lq   1/1     Running   1          5d
nginx-deployment-66b6c48dd5-jv8zh   1/1     Running   1          5d
NAME                                READY   STATUS    RESTARTS   AGE
nginx-deployment-66b6c48dd5-8z2j9   1/1     Running   1          5d
nginx-deployment-66b6c48dd5-bd2lq   1/1     Running   1          5d
nginx-deployment-66b6c48dd5-jv8zh   1/1     Running   1          5d


: 1

- `NAME` - specifies name of the resource; __can be ommitted, if so, will return everything__ (e.g. `kubectl get pods` as seen above)

We might:
- specify multiple `names` __of same `TYPE`__, e.g.:

In [5]:
kubectl get pods nginx-deployment-66b6c48dd5-8z2j9 nginx-deployment-66b6c48dd5-bd2lq

NAME                                READY   STATUS    RESTARTS   AGE
nginx-deployment-66b6c48dd5-8z2j9   1/1     Running   1          5d
nginx-deployment-66b6c48dd5-bd2lq   1/1     Running   1          5d


: 1

- If we have multiple resources __of different `TYPE`s__ we use __`TYPE`/`name` `ANOTHER_TYPE`/`name`__ and so on:

In [6]:
kubectl get pod/example-pod1 replicationcontroller/example-rc1

Error from server (NotFound): pods "example-pod1" not found
Error from server (NotFound): replicationcontrollers "example-rc1" not found


: 1

- Find `POD`s for specific config files:

In [7]:
kubectl get -f ./config/pod.yaml -f ./config/service.yaml

the path "./config/pod.yaml" does not exist
the path "./config/service.yaml" does not exist


: 1

## Operations

Thing to note before moving on:

> ### Prefer to use `.yaml` configs to `kubectl` ad-hoc commands wherever possible!

hence: 

> __Below the most useful ones are provided, full list can be seen [here](https://kubernetes.io/docs/reference/kubectl/overview/#in-cluster-authentication-and-namespace-overrides)__

For example:

- `kubectl annotate` could be described via `.yaml` config for specific `type`
- `kubectl create` should not be used, `kubectl apply` and declarative config files are prefferred

## Core commands

### apply

> __Apply configuration change to a `resource`__

Using this command, after starting `cluster` (e.g. with `minikube` for local development or `kubeadm` for real deployment) __we can configure the cluster according to our liking__.

> This is the preferred way of managment

Important flags we will often use:
- `-R` - __recursively read `config` files within a directory__
- `-f` - filename (__or directory, usually__) to read `config` from

If you would like to use [`kustomize`](https://kustomize.io/) `-k` flag would also come in handy.

### diff

> __Check what changes will be applied to `cluster` DESIRED STATE__

> ### One should always run this command before `apply`ing configuration!

```bash
kubectl diff -f -R FILENAME [flags]
```

Same flags are useful as in the `apply` case.

### get

> __List one or more resources__

Allows us to get basic information about `k8s` objects.

```bash
kubectl get (-f FILENAME | TYPE [NAME | /NAME | -l label]) [--watch] [--sort-by=FIELD] [[-o | --output]=OUTPUT_FORMAT] [flags]
```

### describe

> __Display state of one or more resources__

Another important command, allows us to easily get an overview of the resource(s)

```bash
kubectl describe (-f FILENAME | TYPE [NAME_PREFIX | /NAME | -l label]) [flags]
```

As previously:
- One can specify `file`(s) which describe desired state __in order to get the current one__
- We can specify `TYPE`, `NAME` and other identifying resources for this command

### top

> __Displays resource usage of `Node`(s) or `POD`(s)__

```bash
kubectl top nodes
```

> For `top` to work with `minikube` (not `kubeadm`) please refer to [this StackOverflow answer](https://stackoverflow.com/a/52709663/10886420) and read more about __Metrics API__ if you find it insufficient

## Helpful in debugging commands


### attach

> __Attach to specific running container to read it's `stdout` and/or interact with `stdin`__

```bash
kubectl attach POD -c CONTAINER [-i] [-t] [flags]
```

> Works just like `docker attach`, __useful for debugging__

### cp

> __Copy files from to/from containers__

```bash
kubectl cp <file-spec-src> <file-spec-dest> [options]
```

> Useful for debugging to disk output of container(s)

### exec

> __Exec command against `container` within `POD`__

```bash
kubectl exec POD [-c CONTAINER] [-i] [-t] [flags] [-- COMMAND [args...]]
```

> Useful for debugging container state

## Resources

> One can specify different resources to query

Usually these are `POD`s, `Node`s and `Volume`s but the list is obviously way larger.

See [here](https://kubernetes.io/docs/reference/kubectl/overview/#resource-types) for all available resources

# Selectors

## Field selectors

> __We can select data based on various `fields` (like `status` or `metadata`)__

They work essentially as filters, __we just have to specify `--field-selector` flag and appropriate arguments__:

In [9]:
kubectl get pods --field-selector status.phase==Running

NAME                                READY   STATUS    RESTARTS   AGE
nginx-deployment-66b6c48dd5-8z2j9   1/1     Running   1          5d1h
nginx-deployment-66b6c48dd5-bd2lq   1/1     Running   1          5d1h
nginx-deployment-66b6c48dd5-jv8zh   1/1     Running   1          5d1h


: 1

We can use the following comparison operators:
- `=` (or `==`, should be more readable)
- `!=`

__This is where `namespace`s are immensely useful__

For example:

In [10]:
kubectl get services --field-selector metadata.namespace!=default

No resources found in default namespace.


: 1

We can also chain our `field selector`s.

> This means both selections are `AND`ed together

In [ ]:
kubectl get pods --field-selector=status.phase!=Running,spec.restartPolicy=Always

Another option, for multiple resources:

In [ ]:
kubectl get statefulsets,services --all-namespaces --field-selector metadata.namespace!=default

## Label selectors

> Allows us to grab a set of objects via their defined `labels`

Two types of selectors are available in this case:
- `equality based` (similar to what we've seen above)
- `set based` (__being one of__)

First one, would be (`--selector` and `-l` are equivalent):

In [ ]:
kubectl get pods --selector environment=production,tier=frontend

and it's `set` counterpart (__we can add more `labels` for matching within `()` brackets!__):

In [ ]:
kubectl get pods -l 'environment in (production),tier in (frontend)'

Negative matching is also supported (all `environments` __except__ `frontend`):

In [ ]:
kubectl get pods -l 'environment,environment notin (frontend)'

## Cheat-sheet

> `k8s` provides cheat sheets for common tasks and apps one can use

One can find them [here](https://kubernetes.io/docs/reference/kubectl/cheatsheet/#kubectl-context-and-configuration)

> ### Take 10 minutes to go through this list and make sure you understand __all of the options__

(if not, check `--help` of specific `kubectl` commands, __if this doesn't help ask the instructor!__)

# Challenges

## Mandatory

- Configure autocompletion using `kubectl completion` and putting the contents inside your `.bashrc` (or `.zshrc` or other shell respective config). Check out [this guide](https://kubernetes.io/docs/reference/kubectl/cheatsheet/#kubectl-autocomplete) if you need help.
- Check out [Set References API](https://kubernetes.io/docs/concepts/overview/working-with-objects/labels/#set-references-in-api-objects). What these are, why are they useful?

## Additional

- Check out how [`kubectl`](https://kubernetes.io/docs/reference/kubectl/overview/#in-cluster-authentication-and-namespace-overrides) chooses namespaces and determines it's location
- How to configure output of `kubectl` according to our liking? Check [this section](https://kubernetes.io/docs/reference/kubectl/overview/#output-options) for more information
- Create appropriate aliases for common `kubectl` operations